In [ ]:
:sccache 1
// Skip this or run `cargo install sccache`

In [2]:
:dep z3tracer
// Or use `:dep z3tracer = { path = ".." }` for the local version.

In [3]:
:dep plotters = { default_features = false, features = ["evcxr", "line_series"] }

In [4]:
:efmt {:?}

Error format: {:?} (errors must implement std::fmt::Debug)


In [5]:
:opt 3

Optimization: 3


In [6]:
use z3tracer::{syntax::*, model::*};
use std::collections::*;
use std::str::FromStr;
use plotters::prelude::*;
use plotters::evcxr::SVGWrapper;

In [7]:
:last_error_json
// BUG WORKAROUND: In case of mysteriously truncated error 'consider importing this struct', run the command above again
// to print a detailed error and find the import that is actually missing.
//
// See also :help for more useful commands

Error: 

In [8]:
/// Helper function to process files.
fn process_file(path: &str) -> std::io::Result<Model> {
    let file = std::io::BufReader::new(std::fs::File::open(path)?);
    // Inject non-default configurations here with Model::new(config).
    let mut model = Model::default();
    if let Err(le) = model.process(Some(path.to_string()), file) {
        println!("Error at {:?}: {:?}", le.position, le.error);
    }
    Ok(model)
}

/// Helper trait to print es by their id.
trait ModelExt {
    fn id2s(&self, id: &Ident) -> String;    
}

impl ModelExt for Model {
    fn id2s(&self, id: &Ident) -> String {
        self.id_to_sexp(&BTreeMap::new(), id).unwrap()
    }        
}

In [9]:
// Process some input file obtained with `z3 trace=true proof=true ..`
let model = process_file("../tests/data/file1.log")?;

In [10]:
// Compute top instantiated terms.
let mut top = model.most_instantiated_terms();
top.len()

46

In [11]:
// Retrieve the "timestamps" at which instantiations occur for each of the top terms.
let instantiation_times = top.iter().map(|(_count, id)| {
    let mut timestamps = model.term_data(id).unwrap().instantiations.iter().filter_map(|key| {
        let qi = model.instantiations().get(key).unwrap();
        qi.data.as_ref().map(|d| model.term_data(&d.term).unwrap().timestamp)
    }).collect::<Vec<_>>();
    timestamps.sort();
    let name = match model.term(id).unwrap() { Term::Quant { name, .. } => name, _ => "??" };
    (name.to_string(), timestamps)
}).collect::<Vec<_>>();

In [12]:
// Maximum timestamp (i.e. currently, the number of lines in the Z3 log)
let max_ts = model.processed_logs();
max_ts

443166

In [13]:
// Maximum number of instantiations for the top term.
let max_count = top.peek().unwrap().0;
max_count

359

In [14]:
let figure = evcxr_figure((800, 600), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Top 10 Quantifiers Instantiations", ("Arial", 30).into_font())
        .margin(10)
        .x_label_area_size(40)
        .y_label_area_size(50)
        .build_cartesian_2d(0..max_ts, 0..max_count)?;

    chart.configure_mesh().y_desc("Cumulative number of instantiations").x_desc("Time (line number)").draw()?;

    for (j, (name, values)) in instantiation_times.iter().take(10).enumerate() {
        let color : PaletteColor<Palette9999> = PaletteColor::pick(j);
        chart
        .draw_series(
            LineSeries::new(values.iter().enumerate().map(|(i, c)| (*c, i)), &color)
        ).unwrap()
        .label(name)
        .legend(move |(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], color.filled()));
    }

    chart.configure_series_labels()
        .background_style(&WHITE.mix(0.8))
        .border_style(&BLACK)
        .position(SeriesLabelPosition::UpperLeft)
        .draw()?;
    Ok(())
});
figure

Top 10 Quantifiers Instantiations
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Cumulative number of instantiations
 
 
Time (line number)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
350
 
 
 
 
0
 
 
 
50000
 
 
 
100000
 
 
 
150000
 
 
 
200000
 
 
 
250000
 
 
 
300000
 
 
 
350000
 
 
 
400000
 
 
<polyline fill="none" opacity="1" stroke="#FFE119" stroke-width="1" points="213,549 214,547 214,546 241,544 242,543 243,541 245,540 245,539 246,537 249,536 250,534 252,533 254,532 254,530 255,529 255,527 280,526 281,525 281,523 285,522 285,520 285,519 286,518 289,516 289,515 315,513 316,512 317,511 320,509 322,508 322,506 322,505 323,503 323,502 325,501 325,499 325,498 326,496 327,495 327,494 328,492 329,491 329,489 330,488 330,487 330,485 331,484 331,482 331,481 364,480 365,478 365,477 365,475 365,474 366,473 366,471 367,470 367,468 367,467 367,466 367,464 367,463 367,461 367,460 367,458 367,457 367,456 367,454 367,453 367,451 368,450 368,449 369,447 369,446 369,444 370,443 370,442 370,440 370,439 370,437 370,436 370,435 370,433 370,432 370,430 370,429 370,428 371,426 371,425 371,423 372,422 372,420 373,419 374,418 374,416 375,415 375,413 375,412 390,411 390,409 390,408 391,406 391,405 391,404 392,402 392,401 392,399 392,398 392,397 392,395 392,394 392,392 397,391 397,390 398,388 414,387 414,385 415,384 416,383 417,381 418,380 431,378 432,377 432,375 433,374 434,373 442,371 442,370 445,368 445,367 447,366 448,364 449,363 450,361 450,360 450,359 450,357 459,356 463,354 463,353 463,352 471,350 471,349 471,347 472,346 472,345 472,343 473,342 473,340 474,339 478,337 479,336 493,335 493,333 495,332 497,330 498,329 499,328 499,326 499,325 503,323 504,322 504,321 504,319 508,318 509,316 509,315 509,314 511,312 511,311 514,309 514,308 524,307 525,305 526,304 527,302 527,301 528,300 528,298 528,297 529,295 529,294 529,292 529,291 529,290 529,288 529,287 530,285 530,284 531,283 531,281 531,280 532,278 534,277 534,276 534,274 535,273 535,271 535,270 536,269 536,267 537,266 537,264 537,263 538,262 538,260 538,259 551,257 551,256 552,255 552,253 552,252 552,250 552,249 552,247 552,246 552,245 553,243 553,242 553,240 553,239 553,238 553,236 553,235 553,233 553,232 557,231 557,229 558,228 558,226 558,225 558,224 572,222 572,221 573,219 574,218 574,217 574,215 577,214 580,212 580,211 581,209 581,208 582,207 582,205 583,204 584,202 584,201 585,200 585,198 594,197 595,195 596,194 596,193 597,191 597,190 597,188 598,187 609,186 609,184 611,183 611,181 611,180 611,179 619,177 619,176 624,174 624,173 625,172 625,170 632,169 632,167 636,166 637,164 637,163 638,162 638,160 639,159 640,157 641,156 641,155 642,153 655,152 656,150 656,149 656,148 657,146 658,145 673,143 673,142 674,141 677,139 679,138 679,136 679,135 680,134 696,132 696,131 697,129 697,128 697,126 697,125 698,124 714,122 717,121 720,119 721,118 721,117 735,115 736,114 736,112 736,111 737,110 738,108 739,107 739,105 752,104 758,103 758,101 759,100 759,98 772,97 774,96 774,94 774,93 774,91 775,90 785,89 785,87 "/>
<polyline fill="none" opacity="1" stroke="#0082C8" stroke-width="1" points="213,549 213,547 214,546 240,544 241,543 242,541 245,540 245,539 245,537 249,536 252,534 253,533 254,532 254,530 255,529 280,527 280,526 281,525 284,523 285,522 285,520 285,519 288,518 288,516 289,515 315,513 316,512 316,511 318,509 319,508 321,506 322,505 322,503 322,502 323,501 325,499 325,498 325,496 325,495 326,494 326,492 327,491 328,489 329,488 329,487 330,485 330,484 330,482 331,481 364,480 364,478 365,477 365,475 366,474 366,473 366,471 367,470 367,468 367,467 368,466 368,464 368,463 368,461 368,460 368,458 368,457 368,456 368,454 368,453 368,451 368,450 368,44

In [15]:
// Top instantiated quantified term.
model.id2s(&top.peek()?.1)

"(QUANT ((v2 T@$Value) (v1 T@$Value)) (! (not (or (not (or ($IsEqual_stratified v1 v2) (not (or (= v1 v2) (not (or (not (= (l#$ValueArray (v#$Vector v1)) (l#$ValueArray (v#$Vector v2)))) (not (is v1)) (not (is v2)) (not (or ($IsEqual_level1 (Select_[$int]$Value (v#$ValueArray (v#$Vector v1)) (i!8!0 v2 v1)) (Select_[$int]$Value (v#$ValueArray (v#$Vector v2)) (i!8!0 v2 v1))) (<= (+ (l#$ValueArray (v#$Vector v1)) (* (- 1) (i!8!0 v2 v1))) 0) (not (>= (i!8!0 v2 v1) 0)))))))))) (not (or (= v1 v2) (not ($IsEqual_stratified v1 v2)) (not (or (not (= (l#$ValueArray (v#$Vector v1)) (l#$ValueArray (v#$Vector v2)))) (not (is v1)) (not (is v2)) (not (QUANT ((i Int)) (! (or ($IsEqual_level1 (Select_[$int]$Value (v#$ValueArray (v#$Vector _2)) i) (Select_[$int]$Value (v#$ValueArray (v#$Vector v1)) i)) (>= (+ i (* (- 1) (l#$ValueArray (v#$Vector _2)))) 0) (not (>= i 0))) :qid outputbpl.315:14 :pattern (pattern (Select_[$int]$Value (v#$ValueArray (v#$Vector _2)) i)) :pattern (pattern (Select_[$int]$Value

In [16]:
// Final counter and `Ident` value for the top term.
top.peek()?

(359, #4429!9)

In [17]:
// Introspect a particular term, given by handle.
// Note that the version number `!9` was added during parsing for disambiguation and is not present in the original log file.
model.term(&Ident::from_str("#4429!9")?)?

Quant { name: "outputbpl.310:31", params: 2, triggers: [#284], body: #4443, var_names: Some([VarName { name: Symbol("v2"), sort: Symbol("T@$Value") }, VarName { name: Symbol("v1"), sort: Symbol("T@$Value") }]) }